In [1]:
import os
import sys
import random
import numpy as np
from tqdm import tqdm_notebook
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process.kernels import RBF, WhiteKernel, ConstantKernel
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import ttest_ind
from matplotlib import pyplot as plt

In [2]:
from catboost import CatBoost, Pool

In [3]:
path = '/home/trofim/NAS-Bench-201-v1_1-096897.pth'

from nas_201_api import NASBench201API as API
# Create an API without the verbose log
api = API(path, verbose=False)

In [4]:
import pickle
obj = pickle.load(open('nasbench_val.pkl', 'rb'))

In [13]:
hf_val = []
hf_test = []
hf_times = []

lf_val = []
lf_test = []
lf_times = []

for index in range(len(api)):
    #
    # 200 iter
    #
    info = api.query_meta_info_by_index(index, hp = '200')  # This is an instance of `ArchResults`
    res_metrics = info.get_metrics('cifar10-valid', 'x-valid', 199) # This is a dict with metric names as keys
    hf_val.append(res_metrics['accuracy'])
    res_metrics = info.get_metrics('cifar10', 'ori-test', 199) # This is a dict with metric names as keys
    hf_test.append(res_metrics['accuracy'])
    
    res_metrics = info.get_metrics('cifar10-valid', 'train', 199) # This is a dict with metric names as keys
    hf_times.append(res_metrics['all_time'])
    
    #
    # 12 iter
    #
    info = api.query_meta_info_by_index(index, hp = '12')  # This is an instance of `ArchResults`
    res_metrics = info.get_metrics('cifar10-valid', 'x-valid', 11) # This is a dict with metric names as keys
    lf_val.append(res_metrics['accuracy'])
    res_metrics = info.get_metrics('cifar10', 'ori-test', 11) # This is a dict with metric names as keys
    lf_test.append(res_metrics['accuracy'])
    
    res_metrics = info.get_metrics('cifar10-valid', 'train', 11) # This is a dict with metric names as keys
    lf_times.append(res_metrics['all_time'])

In [209]:
def print_stat(lf, hf):
    s1 = stats.kendalltau(lf, hf).correlation
    s2 = stats.spearmanr(lf, hf).correlation
    s3 = stats.pearsonr(lf, hf)[0]

    return [s1, s2, s3]

In [14]:
max(hf_val), max(hf_test)

(91.60666665039064, 94.37333333333333)

In [81]:
len(list(filter(lambda x : x > 93.96, hf_test)))/len(hf_test)*100

0.30720000000000003

In [391]:
lf_ep1_val = []

for index in range(len(api)):    
    info = api.query_meta_info_by_index(index, hp = '12')  # This is an instance of `ArchResults`
    res_metrics = info.get_metrics('cifar10-valid', 'x-valid', 0) # This is a dict with metric names as keys
    lf_ep1_val.append(res_metrics['accuracy'])

In [392]:
pickle.dump(lf_ep1_val, open('lf_ep1_val_c10.pickle', 'wb'))

In [405]:
print_stat(lf_ep1_val, hf_test)

[0.160959436666964, 0.23530634003795667, 0.6719242274764118]

In [406]:
print_stat(lf_nst_ep1, hf_test) 

[0.4721006059101123, 0.6532957791513205, 0.46699173790688836]

In [16]:
pickle.dump((lf_val, lf_test, hf_val, hf_test), open('acc_nb201_c10.pickle', 'wb'))